## Outils pour la manipulation d'images et librairies.


In [46]:
import PIL
from PIL import Image
import numpy as np
import scipy as sp
import os
from math import log10, sqrt

def load(filename):
    toLoad= Image.open(filename)
    return np.asarray(toLoad)


def psnr(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

def dct2(a):
    return sp.fft.dct( sp.fft.dct( a, axis=0, norm='ortho' ), axis=1, norm='ortho' )

def idct2(a):
    return sp.fft.idct( sp.fft.idct( a, axis=0 , norm='ortho'), axis=1 , norm='ortho')

def save(a):
    Image.fromarray(a).save('./test1.png')
    



## Normalisation de l'image (YCbCr et padding)

In [71]:
def to_Y_Cb_Cr(filename): 
    image = load(filename)
    Y = 0.299*image[:, :, 0] + 0.587*image[:, :, 1] + 0.114*image[:, :, 2]
    Cb = -0.1687*image[:, :, 0] - 0.3313*image[:, :, 1] + 0.5*image[:, :, 2] + 128
    Cr = 0.5*image[:, :, 0] - 0.4187*image[:, :, 1] - 0.0813*image[:, :, 2] +128
    return Y, Cb, Cr # choisir float (+info) ou int(+compression)? plutot int car apres enlever des 0?

In [72]:
def to_RGB(Y, Cb, Cr): 
    R = np.array(np.rint(Y + 1.402*(Cr-128)), dtype='uint8')
    G = np.array(np.rint(Y - 0.34414*(Cb-128) - 0.71414*(Cr-128)), dtype='uint8')
    B = np.array(np.rint(Y + 1.772*(Cb-128)), dtype='uint8')
    image = np.dstack((R,G, B))

    return image

In [81]:
original = load('./test.png')
y, cb, cr = to_Y_Cb_Cr('./test.png')
compressed = to_RGB(y, cb, cr)
#save(compressed)

def psnr(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

print(psnr(original, compressed))

ZeroDivisionError: float division by zero

## Découpage en blocs et compression

## Écriture dans un fichier

## Décompression

## Tests 

In [4]:
test = load("test.png")
Image.fromarray(test,'RGB').show()
